In [57]:
import json
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.spatial.distance
#import nltk

SyntaxError: invalid syntax (<ipython-input-4-554438f9eec4>, line 1)

In [6]:
## Import data
with open('face-rating-data.json') as desc_file:
    descr = json.load(desc_file)
with open('guess-who-data.json') as data_file:    
    data = json.load(data_file)

In [7]:
qa = np.genfromtxt("uid_qa.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U128'),])

In [61]:
#qa[971]

In [8]:
fe = np.genfromtxt("uid_pre_elim.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U256'),])

In [9]:
fe.dtype.names = ('uniqueID', 'bn', 'qn', 'pre_que', 'curr_elim')

#fe.dtype.names
#fe[971]

In [10]:
ftd = np.genfromtxt("face_id_descr.txt", delimiter = ";", skip_header = 1 , usecols = np.arange(0,2), dtype = [('U16'), ('U2056')])

In [11]:
ftd.dtype.names = ('img_id', 'description')
len(ftd['img_id'])
len(set(ftd['img_id']))

191

In [123]:
## first convert strings to list of rows equal to rows in fe
def mk_list(str_vector):
    des_byid = str_vector
    all_x = [i.split() for i in des_byid]
    #for i in des_byid:
    #    j = i.split()
        #if len(j) > 0:
    #    all_x.append(j)

    return all_x


In [131]:
all_x = mk_list(fe['pre_que'])
#print(len(all_x)[1])
print(all_x[0])

['IMG_9048', 'IMG_0214', 'IMG_0401', 'IMG_7806', 'IMG_7942', 'IMG_0356', 'IMG_9514', 'IMG_9957', 'IMG_0566', 'IMG_7882', 'IMG_0641', 'IMG_9364', 'IMG_9634', 'IMG_7700', 'IMG_9710', 'IMG_0701']


In [115]:
## Now for each row in the list above, find description and add it
all_i_txt = []

for rown in range(len(all_x)):
    #if rown not in bl:
        #print(all_x.index)
    int_i_txt = []
    for img in all_x[rown]:
        if img in ftd['img_id']:
            d = str(ftd['description'][list(ftd['img_id']).index(img)])
            int_i_txt.append(str(d))
    all_i_txt.append(int_i_txt)
        
            
            

In [116]:
#print check
#print(type(all_i_txt[0]))
print(len(all_i_txt))
print(all_i_txt[971])

3314
[]


In [117]:
## add question and answer to each data point
for it in range(len(all_i_txt)):
    all_i_txt[it].extend([qa['que'][it].strip(), qa['ans'][it].strip()])
    
    

In [89]:
## check addition of above objects
#for i in range(len(all_i_txt)):
#    print(all_i_txt[i][-5:])

In [90]:
#for i in set(bl):
#    print(all_i_txt[i][-2:])

In [91]:
## Segregate indices where answers are yes or No
yes_ind = []
no_ind = []
for i in range(len(all_i_txt)):
    if all_i_txt[i][-1] == 'No':
        all_i_txt_no.append(all_i_txt[i])
        no_ind.append(i)
    else:
        all_i_txt_yes.append(all_i_txt[i])
        yes_ind.append(i)

In [146]:
no_ind[0:5]

[0,
 1,
 2,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 12,
 13,
 17,
 18,
 22,
 25,
 30,
 33,
 35,
 38,
 39,
 45,
 52,
 57,
 59,
 63,
 65,
 66,
 72,
 73,
 78,
 79,
 87,
 88,
 90,
 91,
 92,
 94,
 95,
 96,
 97,
 100,
 101,
 104,
 105,
 107,
 112,
 118,
 120,
 121,
 122,
 123,
 125,
 126,
 128,
 129,
 130,
 132,
 133,
 134,
 136,
 137,
 138,
 140,
 142,
 146,
 147,
 148,
 149,
 151,
 153,
 155,
 156,
 157,
 158,
 162,
 166,
 167,
 168,
 170,
 171,
 172,
 173,
 174,
 177,
 179,
 181,
 184,
 187,
 188,
 189,
 191,
 194,
 196,
 197,
 198,
 200,
 202,
 203,
 208,
 210,
 214,
 215,
 218,
 219,
 221,
 223,
 226,
 227,
 228,
 230,
 231,
 233,
 234,
 238,
 240,
 241,
 242,
 243,
 247,
 249,
 253,
 255,
 258,
 262,
 263,
 265,
 266,
 268,
 270,
 271,
 272,
 275,
 280,
 283,
 288,
 290,
 291,
 294,
 296,
 299,
 300,
 301,
 302,
 303,
 305,
 307,
 308,
 309,
 310,
 312,
 313,
 314,
 315,
 319,
 320,
 321,
 327,
 328,
 332,
 333,
 335,
 336,
 338,
 343,
 345,
 350,
 355,
 357,
 359,
 360,
 363,
 365,
 366,
 368,


In [118]:
print(len(all_i_txt))

3314


In [119]:
#print(all_i_txt[0])
##basic model
tf = TfidfVectorizer()

In [120]:
## all similarities
all_sim = []
for l in range(len(all_i_txt)):
    H = tf.fit_transform(all_i_txt[l])
    sim = cosine_similarity(H)
    all_sim.append(sim)

In [132]:
#similarities[-2] meaning taking only the array for question
all_sim_q = []
for i in range(len(all_sim)):
    all_sim_q.append(all_sim[i][-2][:-2])
print(len(all_sim_q))
#print(all_sim_q[971])
#print(len(all_sim))

3314


In [133]:

all_cheat = mk_list(fe['curr_elim'])


In [104]:
len(all_cheat_yes)
all_cheat_yes[4]

['IMG_9078',
 'IMG_9927',
 'IMG_0325',
 'IMG_9977',
 'IMG_9409',
 'IMG_7851',
 'IMG_8834',
 'IMG_9755']

In [168]:
cheat_code = zip(all_sim_q, all_cheat, [i[-1] for i in all_i_txt])

In [169]:
list(cheat_code)[3]

(array([ 0.08675302,  0.0928023 ,  0.10133633,  0.03039676,  0.03301987,
         0.        ,  0.02481246,  0.13825282]),
 ['IMG_8138', 'IMG_7578', 'IMG_8592', 'IMG_9861', 'IMG_9153'],
 'Yes')

In [162]:
## Sort all the above similarities with argsort which gives indices
## or we should not take top5 and make a threshold above which we will include it in the list of output?
sorted_ind = []

for si, ln, v in cheat_code:
    if v == "Yes":
        y_top = si.argsort()[::-1][:len(ln)]
        print(y_top)
    else:
        y_top = si.argsort()[:len(ln)]
        print(y_top)
    sorted_ind.append(y_top)
#sorted_ind[971]


[ 5  6 15]
[ 3  2 10  1  5  7 12  4  6  9 11  0]
[15 12  0 10  9  7 14  1]
[7 2 1 0 4]
[2]
[0]
[1 0]
[ 6  9  5  2 13  3  4 12  1 10  0  8 11]
[ 0  9  8 15 12  3  6 14]
[4 5 6 3 1 2 7]
[]
[14]
[]
[11]
[12 13  1  7  8  3  4 11  9  2  6  0]
[1]
[14  4 13  0 10  2  8 12]
[4 1 5]
[]
[2 0 4 1]
[]
[]
[]
[ 0 14  3  7  6  5  9 13 15 10  2  4 12  1]
[1]
[0 1 2 3]
[ 4  8 11 10  9  7]
[0 5 4 3]
[]
[]
[0]
[ 8  6  1  3  9  7 15 14 13]
[1 2 4 6 5 3]
[ 5  4  1 12]
[10  9  7  5  3 11  8  6  4  2  1]
[ 0  2  3  5  7  8  9 12]
[1 4 7 3 6 5 2]
[10 14  5 11  2 15 13 12  9  8  7  6]
[3 0 2]
[11 13 14 15  9  4 12  5]
[0 4 6 5 3]
[0 2]
[ 7 11  0 14  6  2  1 15 12  5  9  3]
[1]
[1 2]
[8 7 4]
[11  3  2 12  4  5 10  7  1  8  6]
[]
[0]
[ 9 14 12  6 10  2  3  7  1]
[1 5 6 4 3 2]
[]
[ 7  3  4  9 14 13 15]
[3 1 2 6 4 0]
[1 2]
[ 0  1  9 14  6  5 11 10]
[0]
[2 3]
[2 1]
[0]
[]
[1]
[14  0  1 10 12 13 11  4  2  3]
[1 2 4]
[2]
[1]
[ 2  5 13  1 15  3]
[0 5 9 8 7 6 4]
[0]
[0]
[ 0  3 13  7 15  1  8  6 10  2  4]
[1 2 3]
[0]
[

In [137]:
top_img = zip(all_x, sorted_ind)


In [138]:
## convert to img_id from index to match with actual image ids selcted by participant
def mk_imgid(arr):
    y_pred = []
    for i,j in arr:
        yp = []
        for s in j:
            yp.append(i[s])
        y_pred.append(yp)
    return y_pred
    
    

In [139]:
pred_img = mk_imgid(top_img)

In [123]:
#print(all_cheat[0:3])
#v = mk_imgid(top_img)
#print(w)

In [141]:
acc_in = zip(all_cheat, pred_img)

In [142]:
acc_all = []
for a,p in acc_in:
    #print(a,p)
    if len(a) == 0:
        acc_i = 0
    else:
        acc_i = len(set(a).intersection(set(p)))/len(set(a))
    acc_all.append(acc_i)

In [143]:
#len(pred_img)
print(acc_all)

[0.0, 0.9166666666666666, 0.25, 0.4, 1.0, 0.0, 0.0, 0.9230769230769231, 0.5, 0.8571428571428571, 0, 0.0, 0, 0.0, 0.8333333333333334, 0.0, 0.125, 0.3333333333333333, 0, 0.75, 0, 0, 0, 0.9285714285714286, 1.0, 0.25, 0.6666666666666666, 0.5, 0, 0, 1.0, 0.5555555555555556, 0.8333333333333334, 0.25, 0.9090909090909091, 0.625, 0.8571428571428571, 0.6666666666666666, 0.6666666666666666, 0.75, 0.4, 0.5, 0.6666666666666666, 0.0, 1.0, 0.3333333333333333, 0.8181818181818182, 0, 1.0, 0.4444444444444444, 0.8333333333333334, 0, 0.42857142857142855, 0.8333333333333334, 0.5, 0.375, 0.0, 0.0, 0.0, 0.0, 0, 1.0, 0.6, 0.6666666666666666, 0.0, 0.0, 0.3333333333333333, 0.7142857142857143, 0.0, 0.0, 0.8181818181818182, 0.6666666666666666, 1.0, 0.42857142857142855, 0.3333333333333333, 1.0, 0.5, 0.4, 0.0, 1.0, 0.5, 0.5, 0.8571428571428571, 0.0, 0.4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.25, 0, 0, 0, 0.6, 0.75, 1.0, 0, 0.42857142857142855, 0.0, 0.0, 0.75, 0.8461538461538461, 0.5, 0.6363636363636364, 1.0, 0.5, 0.42857142

In [144]:
np.mean(np.array(acc_all))

0.38729585517077969